In [1]:
import sys, os, random
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx.pauliweb import PauliWeb, compute_pauli_webs

This notebook demonstrates using PyZX's methods for automatically computing and drawing (bounded) Pauli webs. A Pauli web (a.k.a. _correlation surface_) is a coloring of the edges of ZX-diagram with the following properties for every Z spider `v`:

- _all-or-nothing_: either no incident edges or all incident edges of `v` are labelled X or Y
- _parity_: an even number of incident edges of `v` are labelled Y or Z

All X spiders satisfy analogous conditions, swapping the role of X and Z. A _bounded Pauli web_ is a Pauli web with a chosen set of spiders, called the _boundary_ where the parity condition is allowed to be violated.

We say a spider `anti-commutes` with a Pauli web if it is touching an edge of a different colour.

The function `compute_pauli_webs` automitically associates each non-input vertex `v` in the diagram to an integer `order[v]` giving a time-ordering, and one or two bounded Pauli webs, with the following properties:
1. the boundary of the web consists of only `v` itself and inputs
2. Z spiders have a web `zweb[v]` with a Z-colored edge incident to `v`
3. X spiders have a web `xweb[v]` with a X-colored edge incident to `v`
4. output vertices have two webs `zweb[v]` and `xweb[v]` corresponding to both colors at `v`
5. for non-Pauli spiders `v, w`, if `v` anti-commutes with the Pauli web of `w` then `order[v] < order[w]`

Under the hood, this is using PyZX's gflow-finding algorithm to compute a focussed gflow and translate this data into Pauli webs.

# CNOT Examples

In [2]:
# Here's a simple example of a single CNOT gate

c = zx.qasm("""
qreg q[2];
cx q[0], q[1];
""")
g = c.to_graph()
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True)


In [3]:
# There are 4 interesting Pauli webs corresponding to the two outputs
zx.draw(g, labels=True, pauli_web=xwebs[4])
zx.draw(g, labels=True, pauli_web=xwebs[5])
zx.draw(g, labels=True, pauli_web=zwebs[4])
zx.draw(g, labels=True, pauli_web=zwebs[5])

In [4]:
# The CNOT example also has 2 more bounded Pauli webs, with boundaries on the two interior spiders.
# Since we aren't injecting magic states at these locations, we don't need these Pauli webs.
zx.draw(g, labels=True, pauli_web=zwebs[3])
zx.draw(g, labels=True, pauli_web=xwebs[2])

In [5]:
# Here's an example with more CNOT gates

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
cx q[0], q[2];
cx q[2], q[1];
cx q[0], q[1];
cx q[2], q[0];
""")
g = c.to_graph()

zx.draw(g, labels=True)

In [6]:
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[15])
zx.draw(g, labels=True, pauli_web=zwebs[15])
zx.draw(g, labels=True, pauli_web=xwebs[16])
zx.draw(g, labels=True, pauli_web=zwebs[16])
zx.draw(g, labels=True, pauli_web=xwebs[17])
zx.draw(g, labels=True, pauli_web=zwebs[17])

In [7]:
# Alternatively, we can fuse all the spiders of the same color together to get a
# more interesting diagram.
zx.spider_simp(g)
zx.draw(g, labels=True)

In [8]:
# Since this describes the same unitary, the 6 output webs should all have the same support on
# the inputs as before.
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[15])
zx.draw(g, labels=True, pauli_web=zwebs[15])
zx.draw(g, labels=True, pauli_web=xwebs[16])
zx.draw(g, labels=True, pauli_web=zwebs[16])
zx.draw(g, labels=True, pauli_web=xwebs[17])
zx.draw(g, labels=True, pauli_web=zwebs[17])

# Clifford examples

In [9]:
# Next, we'll look at a single H gate
c = zx.qasm("""
qreg q[1];
h q[0];
""")
g = c.to_graph()

# PyZX renders this as a single H-edge connected to an identity spider
zx.draw(g)

# we don't really need this id-spider, so we can remove it with id_simp
zx.id_simp(g)
zx.draw(g, labels=True)

order, zwebs, xwebs = compute_pauli_webs(g)

In [10]:
# The single output has two Pauli webs
zx.draw(g, labels=True, pauli_web=zwebs[2])
zx.draw(g, labels=True, pauli_web=xwebs[2])

# Note that on a hadamard edge, the color changes in the middle of the edge. To handle this, Pauli webs actually label every
# "half-edge" with a Pauli. (v,w) means the half of the edge touching v, whereas (w,v) means the half of the edge touching w.

#  See for example:
print(zwebs[2].half_edges())

{(2, 0): 'Z', (0, 2): 'X'}


In [11]:
# Here's an example mixing CNOT and H gates

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
h q[1];
h q[2];
cx q[1], q[2];
cx q[0], q[2];
h q[0];
cx q[2], q[1];
h q[1];
h q[1];
cx q[0], q[1];
cx q[2], q[0];
""")
g = c.to_graph()

zx.draw(g, labels=True)

In [12]:
# Note the we get some Y-edges appear, which are shown in a third color
order, zwebs, xwebs = compute_pauli_webs(g)
zx.draw(g, labels=True, pauli_web=xwebs[20])
zx.draw(g, labels=True, pauli_web=zwebs[20])
zx.draw(g, labels=True, pauli_web=xwebs[21])
zx.draw(g, labels=True, pauli_web=zwebs[21])
zx.draw(g, labels=True, pauli_web=xwebs[22])
zx.draw(g, labels=True, pauli_web=zwebs[22])

I'm not sure the best way to handle S gates yet. This seems to depend on how you interpret the Pauli web data. The "MBQC"-style interpretation, which works for arbitrary circuits, says the Pauli web tells us how to push Paulis through the circuit, but some of the phases (and hence the circuit itself) might change. This makes sense if you are choosing the angles on-the-fly, and can vary them based on previous errors. The "stabiliser theory" style, which only works for Clifford circuits, tells us how to push Paulis through the circuit *without* changing it. That is, it visualises the computation of the stabiliser tableau.

# Clifford+T Examples

In [13]:
# Generate a random CNOT, H, T circuit
random.seed(1330)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=3, depth=40)
# for g in c.gates: print(g)
zx.draw(c)

In [14]:
# Convert to a ZX diagram and call the full_reduce procedure on it (PyZX's main ZX diagram optimisation pass)
g = c.to_graph()
zx.full_reduce(g)
zx.to_rg(g)

# Normalise compacts the circuit visually and ensures every input/output is connected to a Z spider
g.normalize()

# Compute the time-ordering on nodes (which is only important for the non-Clifford nodes) and compute the Pauli
# webs for every node.
order, zwebs, xwebs = compute_pauli_webs(g)

# Draw the simplified ZX diagram. Note blue edges correspond to edges with Hadamard gates
zx.draw(g, labels=True)

In [15]:
pw = zwebs[43]
pw.half_edges()

{(16, 5): 'Y',
 (5, 16): 'Y',
 (16, 43): 'Z',
 (43, 16): 'Z',
 (16, 58): 'Z',
 (58, 16): 'X',
 (16, 14): 'Y',
 (14, 16): 'Y',
 (3, 1): 'Z',
 (1, 3): 'Z',
 (3, 43): 'Z',
 (43, 3): 'Z',
 (3, 5): 'Y',
 (5, 3): 'Y',
 (3, 58): 'Z',
 (58, 3): 'X',
 (3, 54): 'Z',
 (54, 3): 'Z',
 (3, 14): 'Y',
 (14, 3): 'Y',
 (5, 2): 'X',
 (2, 5): 'Z',
 (5, 43): 'X',
 (43, 5): 'Z',
 (5, 58): 'X',
 (58, 5): 'X',
 (5, 54): 'X',
 (54, 5): 'Z',
 (14, 58): 'X',
 (58, 14): 'X'}

In [16]:
# Once the Pauli webs have been computed, a specific web can be highlighted by `zx.draw` by passing it in as
# an optional argument. Note that webs change color when they cross Hadamard edges.
zx.draw(g, labels=True, pauli_web=zwebs[43])

We now show how this works in some simpler cases. The first is a single T gate.

The T gate becomes a single, 1-legged phase gadget, connected to the input. This can be implemented by Z-merging a T magic state, then doing either an X or a Y measurement, depending on the parity of the Pauli web.

In [17]:
c = zx.qasm("""
qreg q[1];
t q[0];
""")
zx.draw(c)

g = c.to_graph()
zx.full_reduce(g)
order, zwebs, xwebs = compute_pauli_webs(g)

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=None)

The next example is a circuit with some CNOT gates and a T gate, which can be simplified to a single phase gadget. I'm doing this manually here, since the automated simplifier comes up with a different answer (which is equivalent to this one, up to local Cliffords, but less clear what is going on).

In [18]:
c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[1], q[2];
t q[2];
cx q[1], q[2];
cx q[0], q[1];
""")
zx.draw(c)

# manual ZX simplification to get a single phase gadget
g = c.to_graph()
zx.simplify.gadgetize(g, graphlike=False)
zx.basicrules.strong_comp(g, 5, 7)
zx.simplify.spider_simp(g, quiet=True)
zx.basicrules.strong_comp(g, 3, 6)
zx.simplify.spider_simp(g, quiet=True)
zx.simplify.id_simp(g, quiet=True)

# pauli web calculation
order, zwebs, xwebs = compute_pauli_webs(g)

# highlight the web associated to the T spider
zx.draw(g, labels=True, pauli_web=zwebs[15])